In [1]:
import pandas as pd
import arcpy
from arcgis import GeoAccessor

### Load Cooling Degree Day Data

In [2]:
cdd_csv_path = r'C:\Users\tjjoh\OneDrive\Desktop\GIS 5572\Lab2\cdd.csv'

stem_path = r'C:\Users\tjjoh\Documents\GIS5572'
gdb_name = 'Lab3_GDB'
gdb_path = stem_path + '/' + gdb_name + '.gdb'

fc_path = gdb_path + '/cdd'

In [3]:
#read data into a pandas dataframe
cdd = pd.read_csv(cdd_csv_path)

In [4]:
#convert to a spatially-enabled dataframe
cdd_sedf = GeoAccessor.from_xy(cdd, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

In [5]:
#create a file geodatabase
try:
    arcpy.management.CreateFileGDB(
        out_folder_path = stem_path,
        out_name = gdb_name,
        out_version = 'CURRENT'
    )
except:
    pass

In [6]:
#convert to a feature class
cdd_fc = cdd_sedf.spatial.to_featureclass(location = fc_path)

In [7]:
"""
For interpolation, data needs to be a float or a double. Because cooling degree days is an integer, ArcGIS is unable to
directly interpolate on it. These data management steps add a new double field, copies the data to that field, and
deletes the original integer field.
"""

#list months
months = ['january','february','march','april','may','june','july','august','september','october','november','december']

for month in months:
    #add a new field of type double to attribute table
    arcpy.management.AddFields(
        in_table = cdd_fc,
        field_description = f'temp DOUBLE temp # # #',
        template=None
    )
    
    #copy data to the new field
    arcpy.management.CalculateField(
        in_table = cdd_fc,
        field = 'temp',
        expression = f'!{month}!',
        expression_type = 'PYTHON3',
    )
    
    #delete the original field
    arcpy.management.DeleteField(
        in_table = cdd_fc,
        drop_field = month,
        method = 'DELETE_FIELDS'
    )
    
    #rename the new field to what is was originally called
    arcpy.management.AlterField(
        in_table = cdd_fc,
        field = 'temp',
        new_field_name = month,
        new_field_alias = month,
    )

### Interpolate CDD for each month

In [8]:
for month in months:
    #inverse-distance weighted interpolation
    arcpy.ddd.Idw(
        in_point_features = cdd_fc,
        z_field = month,
        out_raster = gdb_path + f'/{month[:3]}_idw',
        cell_size = 0.0216032,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features = None
    )
    
    #natural neighbor interpolation
    #arcpy.ddd.NaturalNeighbor(
    #    in_point_features = cdd_fc,
    #    z_field = month,
    #    out_raster = gdb_path + f'/{month[:3]}_nn',
    #    cell_size = 0.0216032
    #)
    
    """
    Some months have the same value (0) for all points. Because Kriging relies on a semivariogram, it cannot be used for
    constant or near-constant data. This try-except block allows the script to skip applying Kriging if a semivariogram
    cannot be calculated.
    """
    #try:
        #Kriging interpolation
        #arcpy.ddd.Kriging(
        #    in_point_features = cdd_fc,
        #    z_field = month,
        #    out_surface_raster = gdb_path + f'/{month[:3]}_krig',
        #    semiVariogram_props = 'Spherical # # # #',
        #    cell_size = 0.0216032,
        #    search_radius = 'VARIABLE 12',
        #    out_variance_prediction_raster = None
        #)
    #except:
        #pass

### Load CTU Data

In [9]:
ctu_fc = 'ctus_with_id'

In [10]:
#find the centroid of each CTU
arcpy.management.CalculateGeometryAttributes(
    in_features = ctu_fc,
    geometry_property = 'centx CENTROID_X; centy CENTROID_Y',
    coordinate_system = arcpy.Describe(ctu_fc).spatialReference
)

<Result 'ctus_with_id'>

In [11]:
#create a feature class of the centroidal points
centroids = arcpy.management.XYTableToPoint(
    in_table = ctu_fc,
    out_feature_class = gdb_path + '/centroids',
    x_field = 'centx',
    y_field = 'centy',
    coordinate_system = 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

In [12]:
raster_domain = gdb_path + '/raster_domain'
arcpy.ddd.RasterDomain(
    in_raster = f'{gdb_path}/{months[0][:3]}_idw',
    out_feature_class = raster_domain,
    out_geometry_type = 'POLYGON'
)

selection = arcpy.management.SelectLayerByLocation(
    in_layer = centroids,
    overlap_type = 'COMPLETELY_WITHIN',
    select_features = raster_domain,
    search_distance = None,
    selection_type = 'NEW_SELECTION',
    invert_spatial_relationship = 'INVERT'
)
    
arcpy.edit.Snap(
    in_features = selection,
    snap_environment = f'{raster_domain} EDGE "100 Kilometers"'
)

<Result 'centroids_Layer1'>

In [13]:
#extract the raster value for each CTU (currently uses IDW)
for month in months:
     arcpy.sa.ExtractMultiValuesToPoints(
        in_point_features = centroids,
        in_rasters = f'{gdb_path}/{month[:3]}_idw {month}',
        bilinear_interpolate_values = 'NONE'
    )

In [14]:
for month in months:
    with arcpy.da.UpdateCursor(centroids, [month, 'SHAPE@XY']) as cursor:
        for row in cursor:
            if row[0] is None:
                x, y = row[1]
                

In [15]:
#join the interpolated CDD data with CTUs
arcpy.management.JoinField(
    in_data = ctu_fc,
    in_field = 'UNIQUE_ID',
    join_table = centroids,
    join_field = 'UNIQUE_ID',
    fields = 'january; february; march; april; may; june; july; august; september; october; november; december',
)

<Result 'ctus_with_id'>